# W266 Final Project
### Keane Johnson and Tucker Anderson

##TODO Exploratory Data Analysis

##Baseline Model (KNN)

In [ ]:
from sklearn.naive_bayes import GaussianNB
import pandas as pd
import numpy as np
from pathlib import Path, PosixPath

import gzip
import json

In [2]:
# parse data from compressed json into pandas DF
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [30]:
# #google drive implementation
# from pydrive.auth import GoogleAuth 
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# auth.authenticate_user()
# g_auth = GoogleAuth()
# g_auth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(g_auth)
# my data directory token
# data_token = "'1BiqD6B0Wt5-fiaDvdWSTwkOTK3n_ioTL'"

# file_list = drive.ListFile(
#     {'q': data_token + " in parents and trashed=false"}).GetList()
# # show all files in directory
# for file in file_list:
#   print('title: %s, id: %s' % (file['title'], file['id']))
# book_id = file_list[0]['id']
# book_filename = file_list[0]['title']

# data_download = drive.CreateFile({'id': book_id})
# data_download.GetContentFile(book_filename)

In [3]:
# local implementation
root_path = Path(r"C:\Users\tucka\berkeley\W266\final")
data_path = Path(str(root_path) + "/data")
book_filepath = Path(str(data_path) + "/reviews_Books_5.json.gz")
book_filepath = r"C:\Users\tucka\berkeley\W266\final\data\reviews_Books_5.json.gz"

In [ ]:
book_df = getDF(book_filepath)

In [ ]:
book_df.head

##TODO NN Models